In [1]:
from base_layer_utils import BaseLayerDataRepo, BaseLayerResultsRepo, ModelName
#from base_layer_utils import LightgbmBLE, OneVSOneRegBLE, SklearnBLE, NbSvmBLE, XGBoostBLE, RnnBLE

#from fast_text_data import FastTextDataGenerator
#from tfidf_data import tfidf_data_process

import pandas as pd
import numpy as np
#from sklearn.cross_validation import KFold # replace with model_selection?
from sklearn.model_selection import KFold
import time
import gc
from sklearn.metrics import roc_auc_score

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


loading FastText model...
fasttext model loaded. embedding dimemsion: 300


ImportError: cannot import name 'fasttext_data_process'

In [ ]:
PATH = '~/data/toxic/data/'

train = pd.read_csv(PATH + 'cleaned_train.csv')
#train = pd.read_csv('/home/kai/data/wei/Toxic/data/Shiyi_training.csv').fillna('na')

test = pd.read_csv(PATH + 'cleaned_test.csv')

#train = train.head(1000)
#test = test.head(1000)

train_sentence = train['comment_text_cleaned']
test_sentence = test['comment_text_cleaned']

text = pd.concat([train_sentence, test_sentence])

print(train.shape)
print(test.shape)

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

### before we choose which models to assemble, we can do:
#### 1. scatter plot analysis to check the diversity
#### 2. submit to check if the models have similar performance

## Ensembling:

In [ ]:
def combine_layer_oof_per_label(layer1_oof_dict, label):
    x = None
    data_list = layer1_oof_dict[label]
    for i in range(len(data_list)):
        if i == 0:
            x = data_list[0]
        else:
            x = np.concatenate((x, data_list[i]), axis=1)
    return x

### 1. simple blend of two models

result = np.empty((test.shape[0],len(label_cols)))

# mix the first two models
for i, label in enumerate(label_cols):
    x_train = combine_layer_oof_per_label(layer1_oof_train, label)
    x_test = combine_layer_oof_per_label(layer1_oof_test, label)
    for j in range(x_train.shape[1]):
        roc = roc_auc_score(train[label], x_train[:,j])
        print(label, j, roc) # print out roc for meta feature on meta label (which is just the original train label)
    
    roc_scores_of_a_label = []
    alphas = np.linspace(0,1,1001)
    best_roc = 0
    best_alpha = 0
    for alpha in alphas:
        roc = roc_auc_score(train[label], alpha*x_train[:,0] + (1-alpha)*x_train[:,1])
        if roc > best_roc:
            best_roc = roc
            best_alpha = alpha
    
    print(label, best_roc, best_alpha)
    result[:,i] = best_alpha*x_test[:,0] + (1-best_alpha)*x_test[:,1]

submission = pd.read_csv(PATH + 'sample_submission.csv')#.head(1000)
submission[label_cols] = result
sub_id = int(time.time())
print(sub_id)
submission.to_csv('./StackPreds/mixtwo_' + str(sub_id) + '.csv', index=False)

### 2. stacking

In [3]:
base_layer_results_repo = BaseLayerResultsRepo()

load from file


base_layer_results_repo.add_score('ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real',0.9819)

In [4]:
scores = base_layer_results_repo.show_scores()

0.9827	ModelName.NBLSVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9826	ModelName.NBSVM_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9819	ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9818	ModelName.ONELOGREG_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real
0.9815	ModelName.NBLSVC_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9803	ModelName.NBSVM_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9794	ModelName.LGB_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9793	ModelName.LOGREG_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000
0.9786	ModelName.ONESVC_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w
0.9774	ModelName.NBLSVC_tfidf_word_df2_ng(1, 1)_wmf200000
0.9768	ModelName.NBSVM_tfidf_word_df2_ng(1, 1)_wmf200000
0.9765	ModelName.NBLSVC_tfidf_word_df2_ng(1, 2)_wmf200000
0.9761	ModelName.NBSVM_tfidf_word_df2_ng(1, 2)_wmf200000
0.976	ModelName.LOGREG_tfidf_word_df2_ng(1, 1)_wmf200000
0.9752	ModelName.LOGR

In [10]:
ths = [score for _, score in scores][3:10]

In [11]:
ths

[0.9815, 0.9803, 0.9794, 0.9793, 0.9786, 0.9774, 0.9768]

In [29]:
#base_layer_results_repo.add_score('ModelName.ONELOGREG_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real',0.9818)

ModelName.ONELOGREG_wordtfidf_ng13_mf10w_chartfidf_ng25_mf20w_real already existed in the repo. score: 0 update to 0.9818


In [5]:
layer1_oof_train_loaded, layer1_oof_test_loaded, base_layer_est_preds_loaded = base_layer_results_repo.get_results(threshold=0.9793)

In [8]:
len(list(layer1_oof_train_loaded['toxic']))

8

In [12]:
def get_time():
    from datetime import datetime
    from dateutil import tz

    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/New_York')


    utc = datetime.utcnow()

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    est = utc.astimezone(to_zone)
    
    return est.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
dtrain = xgb.DMatrix(x_train, train[label]) # check if train is still in right shape
dtest = xgb.DMatrix(x_test)

In [9]:
n_features = len(list(layer1_oof_train_loaded['toxic']))
print(n_features)

8


In [ ]:
def get_nn_model(n_features, load_model=False, load_model_file=None):

    input = Input(shape=(n_features,))
    
    x = Dense(8, activation='relu')(input)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input, outputs=x)
    
    #pdb.set_trace()
    if (load_model):
        model.load_weights(load_model_file)
        
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


# only to get best rounnds

# xgb random search top N training

In [21]:
xgb_search = pd.read_csv('xgb_search.csv').sort_values(by='avg_auc', ascending=False)

xgb_search.head(3)

,time,id,threshold,num_models,colsample_bytree,lr,max_depth,subsample,gamma,alpha,...,threat_best_round,insult_best_round,identity_hate_best_round,toxic_auc,severe_toxic_auc,obscene_auc,threat_auc,insult_auc,identity_hate_auc,avg_auc
0,2018-03-07 09:10:35,1520431835,0.9793,8,0.8,0.08,3,0.9,2,0,...,98,101,116,0.984185,0.989722,0.994395,0.989384,0.987255,0.986186,0.988521
1,2018-03-07 09:27:15,1520432835,0.9793,8,0.8,0.08,3,0.9,2,0,...,72,50,68,0.984079,0.989608,0.994363,0.989730,0.987202,0.986017,0.988500


In [23]:
for i in range(2):
    
    now = get_time()

    search_id = xgb_search['id'].values[i]
    model_threshold = xgb_search['threshold'].values[i]
    num_models = xgb_search['num_models'].values[i]
    xgb_colsample_bytree = xgb_search['colsample_bytree'].values[i]
    xgb_learning_rate = xgb_search['lr'].values[i]
    xgb_max_depth = xgb_search['max_depth'].values[i]
    xgb_subsample = xgb_search['subsample'].values[i]
    xgb_gamma = xgb_search['gamma'].values[i]
    xgb_alpha = xgb_search['alpha'].values[i]
    xgb_cv_num_round = xgb_search['cv_num_round'].values[i]
    xgb_cv_nfolds = xgb_search['cv_nfolds'].values[i]
    
    # per label based params 
    best_nrounds = {}
    best_nrounds['toxic'] = xgb_search['toxic_best_round'].values[i]
    best_nrounds['severe_toxic'] = xgb_search['severe_toxic_best_round'].values[i]
    best_nrounds['obscene'] = xgb_search['obscene_best_round'].values[i]
    best_nrounds['threat'] = xgb_search['threat_best_round'].values[i]
    best_nrounds['insult'] = xgb_search['insult_best_round'].values[i]
    best_nrounds['identity_hate'] = xgb_search['identity_hate_best_round'].values[i]
    
    scale_pos_weights = {}
    scale_pos_weights['toxic'] = xgb_search['toxic_pos_scale'].values[i]
    scale_pos_weights['severe_toxic'] = xgb_search['severe_toxic_pos_scale'].values[i]
    scale_pos_weights['obscene'] = xgb_search['obscene_pos_scale'].values[i]
    scale_pos_weights['threat'] = xgb_search['threat_pos_scale'].values[i]
    scale_pos_weights['insult'] = xgb_search['insult_pos_scale'].values[i]
    scale_pos_weights['identity_hate'] = xgb_search['identity_hate_pos_scale'].values[i]
    
    metric_dict_fromcsv = {}
    metric_dict_fromcsv['avg_auc'] = xgb_search['avg_auc'].values[i]
    
    layer1_oof_train_loaded, layer1_oof_test_loaded, base_layer_est_preds_loaded = base_layer_results_repo.get_results(threshold=model_threshold)
    gc.collect() 

    xgb_params = {
        'seed': 0,
        'colsample_bytree': xgb_colsample_bytree,
        'silent': 1,
        'subsample': xgb_subsample,
        'learning_rate': xgb_learning_rate,
        'max_depth': xgb_max_depth,
        'gamma': xgb_gamma,
        'alpha': xgb_alpha,
        'nthread': 7,
        'min_child_weight': 1,
        'objective':'binary:logistic',
        'eval_metric':'auc'
    }

    print('time: %s, id: %d, th: %f, num_models: %d, colsample_bytree: %f, lr: %.7f, \
    max_depth: %d, subsample: %f, gamma: %d, alpha: %d, cv_num_round: %d, cv_nfolds: %d,\
    to_pw: %d, s_pw: %d, ob_pw: %d, th_pw: %d, in_pw: %d, ih_pw: %d, to_br: %d, s_br: %d,\
    ob_br: %d, th_br: %d, in_br: %d, ih_br: %d\
            '%(now,search_id,model_threshold,num_models,\
              xgb_colsample_bytree,xgb_learning_rate,\
              xgb_max_depth,xgb_subsample,xgb_gamma,\
              xgb_alpha,xgb_cv_num_round,xgb_cv_nfolds,\
              scale_pos_weights['toxic'],scale_pos_weights['severe_toxic'],\
              scale_pos_weights['obscene'],scale_pos_weights['threat'],\
              scale_pos_weights['insult'],scale_pos_weights['identity_hate'],\
              best_nrounds['toxic'],best_nrounds['severe_toxic'],\
              best_nrounds['obscene'],best_nrounds['threat'],\
              best_nrounds['insult'],best_nrounds['identity_hate']))


    result = np.empty((test.shape[0],len(label_cols)))
    metric_dict = {}

    for i, label in enumerate(label_cols):
        assert train.shape == (159571, 27)
        x_train = combine_layer_oof_per_label(layer1_oof_train_loaded, label)
        x_test = combine_layer_oof_per_label(layer1_oof_test_loaded, label)

        dtrain = xgb.DMatrix(x_train, train[label]) # check if train is still in right shape
        dtest = xgb.DMatrix(x_test)
        
        xgb_params['scale_pos_weight'] = scale_pos_weights[label]

        gbdt = xgb.train(xgb_params, dtrain, best_nrounds[label])

        result[:,i] = gbdt.predict(dtest)#_proba(x_test)[:,1] # if using xgboost sklearn wrapper

    sub_title = 'xgb_topn_'#posweighted_'
    submission = pd.read_csv(PATH + 'sample_submission.csv')
    submission[label_cols] = result
    submission.to_csv('./StackPreds/TopN_XGB/{}_{}_{}.csv'.format(sub_title,metric_dict_fromcsv['avg_auc'],search_id), index=False)
        

time: 2018-03-07 09:44:10, id: 1520431835, th: 0.979300, num_models: 8, colsample_bytree: 0.800000, lr: 0.0800000,     max_depth: 3, subsample: 0.900000, gamma: 2, alpha: 0, cv_num_round: 500, cv_nfolds: 4,    to_pw: 1, s_pw: 1, ob_pw: 1, th_pw: 1, in_pw: 1, ih_pw: 1, to_br: 231, s_br: 114,    ob_br: 103, th_br: 98, in_br: 101, ih_br: 116            
time: 2018-03-07 09:44:21, id: 1520432835, th: 0.979300, num_models: 8, colsample_bytree: 0.800000, lr: 0.0800000,     max_depth: 3, subsample: 0.900000, gamma: 2, alpha: 0, cv_num_round: 500, cv_nfolds: 4,    to_pw: 10, s_pw: 100, ob_pw: 17, th_pw: 333, in_pw: 20, ih_pw: 112, to_br: 185, s_br: 104,    ob_br: 94, th_br: 72, in_br: 50, ih_br: 68            


In [ ]:
submission.head()

In [ ]:
list(base_layer_est_preds)

model_data_id_list = ['ModelName.RNN_rnn_data_001',
'ModelName.NBSVM_tfidf_word_df2_ng(1, 2)_wmf200000',
'ModelName.NBSVM_tfidf_word_df2_ng(1, 1)_wmf200000',
'ModelName.NBSVM_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000',
'ModelName.LGB_tfidf_word_df2_ng(1, 2)_wmf200000',
'ModelName.LGB_tfidf_word_df2_ng(1, 1)_wmf200000',
'ModelName.LGB_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000',
'ModelName.NBLSVC_tfidf_word_df2_ng(1, 2)_wmf200000',
'ModelName.NBLSVC_tfidf_word_df2_ng(1, 1)_wmf200000',
'ModelName.NBLSVC_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000',
'ModelName.LOGREG_tfidf_word_df2_ng(1, 2)_wmf200000',
'ModelName.LOGREG_tfidf_word_df2_ng(1, 1)_wmf200000',
'ModelName.LOGREG_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000']

base_layer_results_repo.add_score('ModelName.NBLSVC_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000', 0.9815)

base_layer_results_repo.add_score('ModelName.NBLSVC_tfidf_word_df2_ng(1, 2)_wmf200000', 0.9765)

base_layer_results_repo.add_score('ModelName.NBLSVC_tfidf_word_df2_ng(1, 1)_wmf200000', 0.9774)

base_layer_results_repo.add_score('ModelName.LOGREG_tfidf_word_df2_ng(1, 1)_wmf200000', 0.9760)

base_layer_results_repo.add_score('ModelName.LGB_tfidf_word_df2_ng(1, 1)_wmf200000', 0.9723)

base_layer_results_repo.add_score('ModelName.NBLSVC_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000', 0.9815)

base_layer_results_repo.add_score('ModelName.NBLSVC_tfidf_word_df2_ng(1, 2)_wmf200000', 0.9765)

base_layer_results_repo.add_score('ModelName.LOGREG_tfidf_word_df2_ng(1, 2)_wmf200000', 0.9752)

base_layer_results_repo.add_score('ModelName.LGB_tfidf_word_df2_ng(1, 2)_wmf200000', 0.9726)

base_layer_results_repo.add_score('ModelName.NBSVM_tfidf_word_df2_ng(1, 2)_wmf200000', 0.9761)

base_layer_results_repo.add_score('ModelName.NBSVM_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000', 0.9803)

base_layer_results_repo.add_score('ModelName.LOGREG_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000', 0.9793)

base_layer_results_repo.add_score('ModelName.LGB_tfidf_wordchar_charmaxdf0.300000_ng(1, 2)_wmf100000_cmf100000', 0.9794)

base_layer_results_repo.add_score('ModelName.NBSVM_tfidf_word_df2_ng(1, 1)_wmf200000', 0.9768)

# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
}